In [1]:
%load_ext autoreload
%autoreload 2

import numpy as np
import scipy.stats as stats
import scipy.special
#graphing
import matplotlib.pyplot as plt
#stats
import statsmodels.api as sm
from statsmodels.base.model import GenericLikelihoodModel

#import testing
import sys
sys.path.append("../")
import vuong_tests2

In [2]:
beta0 = 1.
beta1 = .25

def gen_data(beta0=beta0,beta1=beta1):
    nobs = 1000
    #parameters
    sigma = 1
    
    epsilon = stats.norm.rvs(loc=0,scale=sigma,size=nobs)
    #censor data below x<0?
    x = stats.norm.rvs(loc=5,scale=5,size=nobs)
    y = beta0+ beta1*x + epsilon
    
    #censor
    y[y<=0] = 0
    return y,x,nobs


yn,xn,nobs = gen_data()
print(xn.shape)
print(sm.add_constant(xn).shape)
print(scipy.stats.mode(yn))

(1000,)
(1000, 2)
ModeResult(mode=array([0.]), count=array([98]))


In [3]:
class Tobit(GenericLikelihoodModel):
    
    def __init__(self, *args,ols=False, **kwargs):
        super(Tobit,self).__init__(*args,**kwargs)
        self.ols = ols

    def loglikeobs(self, params):
        y = self.endog
        x = self.exog
        m = 1*(self.endog == 0) #missingness
        
        beta = params
        
        mu_y = np.matmul(x,beta)
        
        pr_y = stats.norm.logpdf( y, loc = mu_y)
        
       
        #if complete case, assign pr missing to all observations...
        pr_m = stats.norm.logcdf( y, loc = mu_y)
        
        #we're done if ols
        if self.ols:
            return pr_y
        else:
            ll = (1-m)*pr_y + m*pr_m
            return ll
        
    def score(self, params):
        y = self.endog
        x = self.exog
        m = 1*(self.endog == 0) #missingness
        m_x = np.repeat(m,x.shape[1]).reshape(x.shape)
        
        if ols: #if OLS use all the data...
            m, m_x = np.ones(y.shape), np.ones(x.shape)
        
        
        b = params

        beta_jac = np.zeros(len(b))
        
        #for censored
        if not cc and not ols: 
            left_stats = (y - np.dot(x, b)) 
            l_pdf = scipy.stats.norm.logpdf(left_stats)
            l_cdf = scipy.stats.norm.logcdf(left_stats)
            left_frac = np.exp(l_pdf - l_cdf)
            beta_left = np.dot(left_frac*m, x*m_x)
            beta_jac -= beta_left
        
        #for non-censored
        mid_stats = (y - np.dot(x, b))
        beta_mid = np.dot(mid_stats*(1-m), x*(1-m_x) )
        beta_jac += beta_mid
        
        # by chain rule, since the expression above is dloglik/dlogsigma
        return beta_jac



model1 =  Tobit(yn,sm.add_constant(xn))
model1_fit = model1.fit(disp=False)
print(model1_fit.summary())

model2 =  Tobit(yn,sm.add_constant(xn),ols=True)
model2_fit = model2.fit(disp=False)
print(model2_fit.summary())

                                Tobit Results                                 
Dep. Variable:                      y   Log-Likelihood:                -1415.1
Model:                          Tobit   AIC:                             2834.
Method:            Maximum Likelihood   BIC:                             2844.
Date:                Sun, 14 Mar 2021                                         
Time:                        19:58:47                                         
No. Observations:                1000                                         
Df Residuals:                     998                                         
Df Model:                           1                                         
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
const          0.9147      0.048     19.155      0.000       0.821       1.008
x1             0.2534      0.007     38.348      0.0

In [4]:
def setup_shi(yn,xn):
    model1 = Tobit(yn,sm.add_constant(xn))
    model1_fit = model1.fit(disp=False)
    ll1 = model1.loglikeobs(model1_fit.params)
    grad1 =  model1.score_obs(model1_fit.params)    
    hess1 = model1.hessian(model1_fit.params)
    k1 = len(model1_fit.params)
    
    #fit logistic values
    model2 = Tobit(yn,sm.add_constant(xn),ols=True)
    model2_fit = model2.fit(disp=False)
    ll2 = model2.loglikeobs(model2_fit.params)
    grad2 =  model2.score_obs(model2_fit.params)    
    hess2 = model2.hessian(model2_fit.params)
    k2 = len(model2_fit.params)
    
    return ll1,grad1,hess1,ll2,k1, grad2,hess2,k2

yn,xn,nobs = gen_data()
ll1,grad1,hess1,ll2,k1, grad2,hess2,k2 = setup_shi(yn,xn)

In [5]:
beta0 = 1.
beta1 = .25

def gen_data(beta0=beta0,beta1=beta1):
    nobs = 1000
    #parameters
    sigma = 1
    
    epsilon = stats.norm.rvs(loc=0,scale=sigma,size=nobs)
    #censor data below x<0?
    x = stats.norm.rvs(loc=5,scale=5,size=nobs)
    y = beta0+ beta1*x + epsilon
    
    #censor
    y[y<=0] = 0
    return y,x,nobs


reg,boot1,boot2, llr,std, omega = vuong_tests2.monte_carlo(20,gen_data,setup_shi)
print("reg: %s, boot1: %s, boot2: %s, llr:%s, std: %s, omega:%s"%(reg,boot1,boot2,llr,std, omega))

shi_result = vuong_tests2.monte_carlo_shi(20,setup_shi,gen_data)
print(shi_result)

reg: [0.85 0.   0.15], boot1: [0.85 0.   0.15], boot2: [0.9 0.  0.1], llr:-1.957579674311536, std: 5.534364646085143, omega:0.17562142632623784
[0.95 0.   0.05]


In [6]:
beta0 = 1.
beta1 = .5

def gen_data(beta0=beta0,beta1=beta1):
    nobs = 1000
    #parameters
    sigma = 2
    
    epsilon = stats.norm.rvs(loc=0,scale=sigma,size=nobs)
    #censor data below x<0?
    x = stats.norm.rvs(loc=5,scale=5,size=nobs)
    y = beta0+ beta1*x + epsilon
    
    #censor
    y[y<=0] = 0
    return y,x,nobs


reg,boot1,boot2, llr,std, omega = vuong_tests2.monte_carlo(20,gen_data,setup_shi)
print("reg: %s, boot1: %s, boot2: %s, llr:%s, std: %s, omega:%s"%(reg,boot1,boot2,llr,std, omega))

shi_result = vuong_tests2.monte_carlo_shi(20,setup_shi,gen_data)
print(shi_result)

reg: [0.95 0.   0.05], boot1: [0.95 0.   0.05], boot2: [0.9  0.05 0.05], llr:1.824454933224636, std: 16.37046722438711, omega:0.512259061169305
[1. 0. 0.]


In [7]:
beta0 = 1.
beta1 = 1.

def gen_data(beta0=beta0,beta1=beta1):
    nobs = 1000
    #parameters
    sigma = 2
    
    epsilon = stats.norm.rvs(loc=0,scale=sigma,size=nobs)
    #censor data below x<0?
    x = stats.norm.rvs(loc=5,scale=5,size=nobs)
    y = beta0+ beta1*x + epsilon
    
    #censor
    y[y<=0] = 0
    return y,x,nobs


reg,boot1,boot2, llr,std, omega = vuong_tests2.monte_carlo(20,gen_data,setup_shi)
print("reg: %s, boot1: %s, boot2: %s, llr:%s, std: %s, omega:%s"%(reg,boot1,boot2,llr,std, omega))

shi_result = vuong_tests2.monte_carlo_shi(20,setup_shi,gen_data)
print(shi_result)

reg: [0. 1. 0.], boot1: [0. 1. 0.], boot2: [0. 1. 0.], llr:343.20418318185574, std: 62.07728362364256, omega:1.9247631689752989
[0. 1. 0.]


In [8]:
def gen_data(beta0=beta0,beta1=1):
    nobs = 1000
    #parameters
    sigma = 1
    
    epsilon = stats.norm.rvs(loc=0,scale=sigma,size=nobs)
    #censor data below x<0?
    x = stats.norm.rvs(loc=5,scale=5,size=nobs)
    y = beta0+ beta1*x + epsilon
    
    #censor
    y[y<=0] = 0
    return y,x,nobs

reg,boot1,boot2, llr,std, omega = vuong_tests2.monte_carlo(20,gen_data,setup_shi)
print("reg: %s, boot1: %s, boot2: %s, llr:%s, std: %s, omega:%s"%(reg,boot1,boot2,llr,std, omega))

shi_result = vuong_tests2.monte_carlo_shi(20,setup_shi,gen_data)
print(shi_result)

reg: [0. 1. 0.], boot1: [0. 1. 0.], boot2: [0. 1. 0.], llr:452.8339019195564, std: 47.299243893618616, omega:2.005188794717262
[0. 1. 0.]


In [9]:
def gen_data(beta0=beta0,beta1=1):
    nobs = 1000
    #parameters
    sigma = 1
    
    epsilon = stats.norm.rvs(loc=0,scale=sigma,size=nobs)
    #censor data below x<0?
    x = stats.norm.rvs(loc=5,scale=5,size=nobs)
    y = beta0+ beta1*x + epsilon
    m = beta0 + epsilon
    #censor
    y[m<=0] = 0
    return y,x,nobs
    
reg,boot1,boot2, llr,std, omega = vuong_tests2.monte_carlo(20,gen_data,setup_shi)
print("reg: %s, boot1: %s, boot2: %s, llr:%s, std: %s, omega:%s"%(reg,boot1,boot2,llr,std, omega))

shi_result = vuong_tests2.monte_carlo_shi(20,setup_shi,gen_data)
print(shi_result)

reg: [0. 0. 1.], boot1: [0. 0. 1.], boot2: [0.35 0.   0.65], llr:-170.52784262197744, std: 34.31888442698855, omega:1.0198149561269951
[0.1 0.  0.9]


In [10]:
beta0 = 1.
beta1 = .25

def gen_data(beta0=beta0,beta1=beta1):
    nobs = 1000
    #parameters
    sigma = 2
    
    epsilon = stats.norm.rvs(loc=0,scale=sigma,size=nobs)
    #censor data below x<0?
    x = stats.norm.rvs(loc=5,scale=5,size=nobs)
    y = beta0+ beta1*x + epsilon
    
    #censor
    y[y<=0] = 0
    return y,x,nobs


reg,boot1,boot2, llr,std, omega = vuong_tests2.monte_carlo(20,gen_data,setup_shi)
print("reg: %s, boot1: %s, boot2: %s, llr:%s, std: %s, omega:%s"%(reg,boot1,boot2,llr,std, omega))

shi_result = vuong_tests2.monte_carlo_shi(20,setup_shi,gen_data)
print(shi_result)

reg: [0. 0. 1.], boot1: [0. 0. 1.], boot2: [0. 0. 1.], llr:-106.99758814711106, std: 9.075687047270325, omega:0.29031434151061175
[0. 0. 1.]


In [11]:
def gen_data(beta0=3.29543322,beta1=0):
    nobs = 1000
    #parameters
    sigma = 1
    
    epsilon = stats.norm.rvs(loc=0,scale=sigma,size=nobs)
    #censor data below x<0?
    x = stats.norm.rvs(loc=5,scale=5,size=nobs)
    y = beta0+ beta1*x + epsilon
    m = beta0 + epsilon
    #censor
    y[m<=0] = 0
    return y,x,nobs
    
reg,boot1,boot2, llr,std, omega = vuong_tests2.monte_carlo(20,gen_data,setup_shi)
print("reg: %s, boot1: %s, boot2: %s, llr:%s, std: %s, omega:%s"%(reg,boot1,boot2,llr,std, omega))

shi_result = vuong_tests2.monte_carlo_shi(20,setup_shi,gen_data)
print(shi_result)

../vuong_tests2.py:74: RuntimeWarning: divide by zero encountered in double_scalars
  boot_teststat =  (llr +V.sum()/(2*nobs) )/(np.sqrt(omega2*nobs))
../vuong_tests2.py:75: RuntimeWarning: divide by zero encountered in double_scalars
  boot_teststat_nml = (llr +V.sum()/(2*nobs) )/(np.sqrt(omega2*nobs))
../vuong_tests2.py:74: RuntimeWarning: divide by zero encountered in cdouble_scalars
  boot_teststat =  (llr +V.sum()/(2*nobs) )/(np.sqrt(omega2*nobs))
../vuong_tests2.py:75: RuntimeWarning: divide by zero encountered in cdouble_scalars
  boot_teststat_nml = (llr +V.sum()/(2*nobs) )/(np.sqrt(omega2*nobs))
/home/erichschulman/anaconda3/lib/python3.7/site-packages/numpy/lib/function_base.py:3948: RuntimeWarning: invalid value encountered in multiply
  x1 = take(ap, indices_below, axis=axis) * weights_below
/home/erichschulman/anaconda3/lib/python3.7/site-packages/numpy/lib/function_base.py:3949: RuntimeWarning: invalid value encountered in multiply
  x2 = take(ap, indices_above, axis=axis

reg: [1. 0. 0.], boot1: [0.95 0.   0.05], boot2: [1. 0. 0.], llr:-0.6339001981230918, std: 0.9389337872189765, omega:0.017464108234663055


../vuong_tests2.py:205: RuntimeWarning: divide by zero encountered in true_divide
  Tnd = (nLR_hat+V.sum()/2)/np.sqrt(n*nomega2_hat + cstar*(V*V).sum())
../vuong_tests2.py:205: RuntimeWarning: divide by zero encountered in true_divide
  Tnd = (nLR_hat+V.sum()/2)/np.sqrt(n*nomega2_hat + cstar*(V*V).sum())
../vuong_tests2.py:205: RuntimeWarning: divide by zero encountered in true_divide
  Tnd = (nLR_hat+V.sum()/2)/np.sqrt(n*nomega2_hat + cstar*(V*V).sum())
../vuong_tests2.py:205: RuntimeWarning: divide by zero encountered in true_divide
  Tnd = (nLR_hat+V.sum()/2)/np.sqrt(n*nomega2_hat + cstar*(V*V).sum())
../vuong_tests2.py:205: RuntimeWarning: divide by zero encountered in true_divide
  Tnd = (nLR_hat+V.sum()/2)/np.sqrt(n*nomega2_hat + cstar*(V*V).sum())
../vuong_tests2.py:205: RuntimeWarning: divide by zero encountered in true_divide
  Tnd = (nLR_hat+V.sum()/2)/np.sqrt(n*nomega2_hat + cstar*(V*V).sum())
../vuong_tests2.py:205: RuntimeWarning: divide by zero encountered in true_divide


[0.2  0.25 0.55]


../vuong_tests2.py:205: RuntimeWarning: divide by zero encountered in true_divide
  Tnd = (nLR_hat+V.sum()/2)/np.sqrt(n*nomega2_hat + cstar*(V*V).sum())
